In [1]:
import glob
import os
import re
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
import nltk

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import f1_score,accuracy_score
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [ ]:
path='C:/Users/Administrator/Desktop/Datathon/datasets-v5/task 2/article'
text=pd.DataFrame()
# filelist=os.path(path)
for files in glob.glob(os.path.join(path,'*.txt')):
    file=open(files,'r',encoding='utf-8')
    for line in file:
        line=re.sub('[^a-zA-Z]'," ",line)
        text_dict={'line':line}
        text=text.append(text_dict,ignore_index=True)

In [50]:
text.to_csv('column.csv',index=False)

In [5]:
path='C:/Users/Administrator/Desktop/Datathon/datasets-v5/test data/test-INPUT/tasks-2-3/test/articles'
text_test=pd.DataFrame()
# filelist=os.path(path)
for files in glob.glob(os.path.join(path,'*.txt')):
    file=open(files,'r',encoding='utf-8')
    for line in file:
        line=re.sub('[^a-zA-Z]'," ",line)
        text_dict={'line':line}
        text_test=text_test.append(text_dict,ignore_index=True)

In [6]:
text_test.to_csv('column for test.csv',index=False)

In [2]:
path='C:/Users/Administrator/'
os.chdir(path)
print(os.getcwd())

C:\Users\Administrator


In [11]:
data_train=pd.read_excel('C:/Users/Administrator/Desktop/Datathon/datasets-v5/task 2/train.xlsx').dropna().drop('labels',axis=1)
data_test=pd.read_excel('C:/Users/Administrator\Desktop/Datathon/datasets-v5/test data/test-INPUT/tasks-2-3/test/test.xlsx').dropna().drop(['Source.Name','Column3'],axis=1)
data_test.head()

,id,line_no,line
0,111111111,1,Next plague outbreak in Madagascar could be s...
1,111111111,2,
2,111111111,3,Geneva The World Health Organisation chief o...
3,111111111,4,
4,111111111,5,The next transmission could be more pronounce...


In [12]:
data_train.head()

,id,line_no,type,lines
0,111111112.0,1.0,non-propaganda,US bloggers banned from entering UK
2,111111112.0,3.0,non-propaganda,Two prominent US bloggers have been banned fro...
4,111111112.0,5.0,propaganda,Pamela Geller and Robert Spencer co-founded an...
6,111111112.0,7.0,non-propaganda,They were due to speak at an English Defence L...
8,111111112.0,9.0,non-propaganda,A government spokesman said individuals whose ...


In [4]:
new_data_train=pd.DataFrame()
new_data_train['id']=data_train['id']
new_data_train['type']=data_train['type']
new_data_train['lines']=data_train['lines'].str.replace('[^a-zA-Z ]','').str.lower()
new_data_train.head()

,id,type,lines
0,111111112.0,non-propaganda,us bloggers banned from entering uk
2,111111112.0,non-propaganda,two prominent us bloggers have been banned fro...
4,111111112.0,propaganda,pamela geller and robert spencer cofounded ant...
6,111111112.0,non-propaganda,they were due to speak at an english defence l...
8,111111112.0,non-propaganda,a government spokesman said individuals whose ...


In [13]:
new_data_test=pd.DataFrame()
new_data_test['id']=data_test['id']
new_data_test['line_no']=data_test['line_no']
new_data_test['lines']=data_test['line'].str.replace('[^a-zA-Z ]','').str.lower()
new_data_test.head()

,id,line_no,lines
0,111111111,1,next plague outbreak in madagascar could be s...
1,111111111,2,
2,111111111,3,geneva the world health organisation chief o...
3,111111111,4,
4,111111111,5,the next transmission could be more pronounce...


In [14]:
lemma=WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words('english')
def clean_data(text):
    words=nltk.word_tokenize(text)
    words=[lemma.lemmatize(word) for word in words if word not in stopwords]
    return " ".join(words)

In [19]:
new_clean_data_train=pd.DataFrame()
for i in range(new_data_train.shape[0]):
    try:
        change_data_train={'lines':str(clean_data(new_data_train.iloc[i]['lines'])),
                           'type':new_data_train.iloc[i]['type'],
                           'id':new_data_train.iloc[i]['id']}
    except:continue
    new_clean_data_train=new_clean_data_train.append(change_data_train,ignore_index=True)

In [20]:
new_clean_data_train.head()

,id,lines,type
0,111111112.0,u blogger banned entering uk,non-propaganda
1,111111112.0,two prominent u blogger banned entering uk hom...,non-propaganda
2,111111112.0,pamela geller robert spencer cofounded antimus...,propaganda
3,111111112.0,due speak english defence league march woolwic...,non-propaganda
4,111111112.0,government spokesman said individual whose pre...,non-propaganda


In [22]:
new_clean_data_test=pd.DataFrame()
for i in range(new_data_test.shape[0]):
    try:
        change_data_test={'lines':str(clean_data(new_data_test.iloc[i]['lines'])),
                           'line_no':new_data_test.iloc[i]['line_no'],
                           'id':new_data_test.iloc[i]['id']}
    except:continue
    new_clean_data_test=new_clean_data_test.append(change_data_test,ignore_index=True)

In [23]:
new_clean_data_test=new_clean_data_test.set_index('id')
new_clean_data_test.head()

,line_no,lines
id,,
111111111.0,1.0,next plague outbreak madagascar could stronger
111111111.0,2.0,
111111111.0,3.0,geneva world health organisation chief wednesd...
111111111.0,4.0,
111111111.0,5.0,next transmission could pronounced stronger di...


In [24]:
# x=new_clean_data.drop('type',axis=1)
# y=new_clean_data['type']
# train_x,test_x,train_y,test_y=train_test_split(x['lines'],y,test_size=0.2,random_state=100)
train_x=new_clean_data_train['lines']
train_y=new_clean_data_train['type']
test_x=new_clean_data_test['lines']

In [25]:
def vector_variable(vector,x1,x2):
    vec=vector()
    return vec.fit_transform(x1), vec.transform(x2)

In [26]:
def get_score(model,x1,y1,x2,y2):
    import warnings
    warnings.filterwarnings("ignore")
    model_name=model
    model=model()
    model.fit(x1,y1)
    pred=model.predict(x2)
    (print('f1_score for ',model_name,' is: %0.3f'%f1_score(pred,y2, average='weighted')))  

In [29]:
cv_train_x,cv_test_x=vector_variable(CountVectorizer,train_x,test_x)

In [30]:
svm=SVC()
svm.fit(cv_train_x,train_y)
pred=svm.predict(cv_test_x)
update_article=pd.DataFrame({'Articles':test_x.index,'Predicted_type':pred})

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [31]:
update_article.head()

,Articles,Predicted_type
0,111111111.0,non-propaganda
1,111111111.0,non-propaganda
2,111111111.0,non-propaganda
3,111111111.0,non-propaganda
4,111111111.0,non-propaganda


In [94]:
# get_score(LogisticRegression,cv_train_x,train_y,cv_test_x,test_y)
# get_score(MultinomialNB,cv_train_x,train_y,cv_test_x,test_y)
# get_score(PassiveAggressiveClassifier,cv_train_x,train_y,cv_test_x,test_y)
# get_score(DecisionTreeClassifier,cv_train_x,train_y,cv_test_x,test_y)
# get_score(RandomForestClassifier,cv_train_x,train_y,cv_test_x,test_y)
# get_score(AdaBoostClassifier,cv_train_x,train_y,cv_test_x,test_y)
# get_score(XGBClassifier,cv_train_x,train_y,cv_test_x,test_y)
# get_score(SVC,cv_train_x,train_y,cv_test_x,test_y)

f1_score for  <class 'sklearn.linear_model.logistic.LogisticRegression'>  is: 0.742
f1_score for  <class 'sklearn.naive_bayes.MultinomialNB'>  is: 0.759
f1_score for  <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'>  is: 0.667
f1_score for  <class 'sklearn.tree.tree.DecisionTreeClassifier'>  is: 0.656
f1_score for  <class 'sklearn.ensemble.forest.RandomForestClassifier'>  is: 0.714
f1_score for  <class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>  is: 0.823
f1_score for  <class 'xgboost.sklearn.XGBClassifier'>  is: 0.848
f1_score for  <class 'sklearn.svm.classes.SVC'>  is: 0.850


In [96]:
# tf_train_x,tf_test_x=vector_variable(TfidfVectorizer,train_x,test_x)

In [97]:
# get_score(LogisticRegression,tf_train_x,train_y,tf_test_x,test_y)
# get_score(MultinomialNB,tf_train_x,train_y,tf_test_x,test_y)
# get_score(PassiveAggressiveClassifier,tf_train_x,train_y,tf_test_x,test_y)
# get_score(DecisionTreeClassifier,tf_train_x,train_y,tf_test_x,test_y)
# get_score(RandomForestClassifier,tf_train_x,train_y,tf_test_x,test_y)
# get_score(AdaBoostClassifier,tf_train_x,train_y,tf_test_x,test_y)
# get_score(XGBClassifier,tf_train_x,train_y,tf_test_x,test_y)
# get_score(SVC,tf_train_x,train_y,tf_test_x,test_y)

f1_score for  <class 'sklearn.linear_model.logistic.LogisticRegression'>  is: 0.843
f1_score for  <class 'sklearn.naive_bayes.MultinomialNB'>  is: 0.847
f1_score for  <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'>  is: 0.686
f1_score for  <class 'sklearn.tree.tree.DecisionTreeClassifier'>  is: 0.658
f1_score for  <class 'sklearn.ensemble.forest.RandomForestClassifier'>  is: 0.770
f1_score for  <class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>  is: 0.812
f1_score for  <class 'xgboost.sklearn.XGBClassifier'>  is: 0.847
f1_score for  <class 'sklearn.svm.classes.SVC'>  is: 0.850
